# Série d'exercice 13 : Poutres avec torsion, symétries

On vous encourage à poser vos questions et donner votre feedback sur ce notebook sur la plateforme ED Discussion du cours accessible en cliquant sur ce bouton:
 
 
 
<div class="container" >
        <a href="https://edstem.org/eu/courses/409/discussion?category=Exercices">
            <button class="btn btn-primary btn-lg">Ed Discussion</button>
        </a>
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
import math as m
from plot import *

## Exercice 1 : Cadres

On considère la structure cadre suivante, soumise à une charge externe $P = 200$ MN comme illustré ci-dessous:

In [ ]:
Image(filename = './Images/Frame.png',  width=350, height=200)

Le cadre est composé de deux poutres. Les propriétés matérielles des deux poutres sont identiques:
  \begin{align*}
    A &= 3\times 10^{-2}\text{ m} \\
    I &= 3\times 10^{-4}\text{ m}^4 \\
    E &= 70 \text{ GPa}
  \end{align*}
Étant donné que la structure présente une symétrie, vous devrez considérer un problème équivalent qui considère une seule poutre.

1. Réduire le cadre présenté à une structure équivalente qui considère une seule poutre. Quelles seront les nouvelles conditions limites et les charges pour ce nouveau problème ?

---

 **Solution:**

 ---



En utilisant la symétrie du problème, le cadre peut être réduit à une seule poutre avec les conditions limites présentés ci-dessous. Une charge nodale vertical de valeur $P/2$ est imposée sur le noeud 1.

Conditions limites: Comme le noeud 0 est encastré, les degrés de libertés associés sont nuls, et comme le noeud 1 est contraint de se déplacer uniquement dans la direction y, seuls les degrés de libertés verticaux sont non nuls:
  \begin{align*}
    &\text{Nœud 0 :}\quad u_0 = 0;\quad v_0 = 0;\quad \phi_0 = 0 \\
    &\text{Nœud 1 :}\quad u_1 = 0;\quad \phi_1 = 0
  \end{align*}

In [ ]:
###########
# Solution:
##########

Image(filename = './Images/Frame_single.png',  width=350, height=200)

2. En réutilisant la fonction _MatriceRigiditePoutreAxiaux_, définie lors de la série 8, formuler la matrice de rigidité $k_1$ pour l'élément 1 dans son axe.

In [ ]:
def MatriceRigiditePoutreAxiaux(E, I, L, A):
    K = (E*I/L**3)*np.array([
        [A*L**2/I, 0, 0, -A*L**2/I, 0, 0],
        [0, 12, 6*L, 0, -12, 6*L],
        [0, 6*L, 4*L**2, 0, -6*L, 2*L**2],
        [-A*L**2/I, 0, 0, A*L**2/I, 0, 0],
        [0, -12, -6*L, 0, 12, -6*L],
        [0, 6*L, 2*L**2, 0, -6*L, 4*L**2]
    ])
    return(K)

In [ ]:
###########
# Solution:
##########

A = 0.03 # [m]
I = 3*10**(-4) # [m4]
E = 70*10**9 # [Pa]
L = np.sqrt(3**2+4**2)

Kl = MatriceRigiditePoutreAxiaux(E, I, L, A) # [N/m]
plot_matrix(Kl, 'K_l')

3. Formuler la matrice de rigidité $K$ exprimée pour les axes globaux. Pour simplifier la suite du problème, définisser une fonction nommée _MatriceRotationPoutre(cos, sin, Kl)_ permettant de calculer la matrice de transformation.

In [ ]:
# Pour la transformation de cette dernière, du système de coordonnée local vers le global, 
# les valeurs de cos θ et sin θ sont données :
cos = 3/L
sin = 4/L

def MatriceRotationPoutre(cos, sin, Kl):
    # La matrice de transformation sera :
    T = np.zeros((6, 6))
    T[:2, :2] = np.array([
        [cos, sin],
        [-sin, cos]
    ])
    T[2, 2] = 1
    T[3:5, 3:5] = np.array([
        [cos, sin],
        [-sin, cos]
    ])
    T[5, 5] = 1
    return T
    
T = MatriceRotationPoutre(cos, sin, Kl)
plot_matrix(T, 'T')

# La matrice de rigidité locale $k_1$ est donc :
Kg = T.T@Kl@T
plot_matrix(Kg, 'Kg')

4. Résoudre le système d'équations.

In [ ]:
###########
# Solution:
##########


# Matrice de raideur réduite au vu des conditions aux limites :
Kred = Kg[4:5, 4:5]

# La seule force connue est f_1y :
P = -200000000 # [N]
f_1y = np.array([P/2])

# La solution pour le déplacement nodal du noeud 1 est :
v1 = np.linalg.inv(Kred)@f_1y
print('v_1 [m] = ', v1)

La fonction plot_strcuture permet de représenter graphiquement le déplacement de la structure. Elle prend en entrée la position des noeuds, la connectivité entre eux ainsi que leur déplacement (u, v, $\theta$). Un exemple vous est présenté ci-dessous.

In [ ]:
positions = np.array([
    [0, 0],
    [3, 4]
])

conn = [
    [0, 1],
]

displacement = [[0, 0, 0], [0, v1[0], 0]]

plot_structure(positions, conn, displacement = displacement)

5. Modifier la force P et observer la nouvelle déformée du cadre

In [ ]:
###########
# Solution:
##########


positions = np.array([
    [0, 0],
    [3, 4]
])

conn = [
    [0, 1],
]

# A modifier :
P = +200000000 # [N]
f_2y = P/2

# La solution pour le déplacement nodal du noeud 2 est :
v2 = np.linalg.inv(Kred)*f_2y
displacement = [[0, 0, 0], [0, float(v2), 0]]

plot_structure(positions, conn, displacement = displacement)

Maintenant considérez cette même structure pour laquelle un déplacement $u_1 = U =1\times 10^{-1}$ m est prescrit sur le noeud 1 dans la direction de l'axe $x$ comme illustré ci-dessous:

In [ ]:
Image(filename = './Images/Frame_Part2.png', width=350, height=200)

L'application d'un déplacement horizontal sur le noeud 1 rompt la symétrie du problème et donc il ne peut plus être réduit. En prenant en compte ce dernier point:

6. Assembler la matrice de rigidité $K$ exprimée pour les axes globaux.

---

 **Solution:**

 ---



Si l'on considère deux éléments poutre, leur matrice de rigidité dans le système de coordonnées local seront égales et de valeur $k_1$. Pour calculer ces dernières dans le système de coordonnées global, les valeurs de $\cos\theta$ et $\sin\theta$ doivent une fois de plus être utilisées:
  \begin{align*}
    &\text{Element 1: } \cos\theta = 3/L \quad \sin\theta = 4/L \\
    &\text{Element 2: } \cos(\theta) = 3/L \quad \sin(-\theta) = -4/L
  \end{align*}

In [ ]:
plot_matrix(Kl, 'Kl')

In [ ]:
###########
# Solution:
##########


# Element 1
cos1 = 3/L
sin1 = 4/L
T1=MatriceRotationPoutre(cos1, sin1, Kl)
Kg1 = T1.T@Kl@T1

# Element 2
cos2 = 3/L
sin2 = -4/L
T2=MatriceRotationPoutre(cos2, sin2, Kl)
Kg2 = T2.T@Kl@T2

# Assemblage
Kgtot = np.zeros((9, 9)) # Matrice de rigidité globale
Kgtot[:6, :6] = Kg1
Kgtot[3:, 3:] = Kgtot[3:, 3:] + Kg2

# Conditions initiales
# Les noeuds 0 et 2 sont encastrées, la matrice de raideur réduite conserve uniquement les ddl du noeud 1

Kgred = Kgtot[3:6, 3:6]

plot_matrix(Kgred, 'Kgred')

7. Assembler le vecteur de force $F$ exprimée pour les axes globaux.

---

 **Solution:**

 ---



Le vecteur de force global $F$ est :
  \begin{equation*}
    F^T = \begin{pmatrix}
      F_{0x} & F_{0y}  &	M_0 & F_{1x} & -P & 0 &F_{2x} & F_{2y} & M_2
    \end{pmatrix}
  \end{equation*}

8. Utiliser le déplacement $u_1$ pour modifier le système d'équation.

---

 **Solution:**

 ---



Le vecteur de déplacement $U$ pour le système est:
  \begin{equation*}
    U^T = \begin{pmatrix}
      0 & 0  &	0 & u_1 & v_1 & \phi_1 & 0 & 0 & 0
    \end{pmatrix}
  \end{equation*}
  Le déplacement prescrit sur le noeud 1 ($u_1=U$) peut être exprimé dans le système d'équation en considérant d'une part la matrice réduite associée aux degrée de libertés libres et d'autre part la contribution de $u_1$ en tant que force équivalente:
  \begin{equation*}
    K_r = \begin{pmatrix}
      K_{55} & K_{56} \\
      K_{65} & K_{66}  \\
    \end{pmatrix}\quad\quad F_r=
    \begin{pmatrix}
      -P\\
      0 \\
    \end{pmatrix}\quad\quad F_{u_1,eq}=
    \begin{pmatrix}
      K_{54}\\
      K_{64} \\
    \end{pmatrix} U
  \end{equation*}
  
  L'équilibre s'écrit:
  \begin{equation*}
    K_r \begin{pmatrix}
      v_1  \\
      \phi_1 \\
    \end{pmatrix} = F - F_{u_1,eq}
  \end{equation*}

In [ ]:
###########
# Solution:
##########


# Matrice de raideur réduite
Kr = Kgtot[4:6, 4:6]
plot_matrix(Kr, 'Kr')

# Le vecteur de force :
P = 200000000 # N
Fr = np.array([[-P], [0]])

U = 1*10**(-1)
Ku2 = np.array([
    [Kgtot[4, 3]],
    [Kgtot[5, 3]]
])
Fu2 = U*Ku2

F_eff = Fr - Fu2

9. Résoudre le système d'équations et tracer la déformée.

In [ ]:
###########
# Solution:
##########

deplacement = np.linalg.inv(Kr)@F_eff
print('Déplacement vertical au noeud 1 [m]', float(deplacement[0].item()))
print('Rotation au noeud 1 [rad]]', float(deplacement[1].item()))

positions = np.array([
    [0, 0],
    [3, 4],
    [6, 0]
])

conn = np.array([
    [0, 1],
    [1, 2]
])

displacement = [[0, 0, 0], [U, float(deplacement[0].item()), float(deplacement[1].item())], [0, 0, 0]]
plot_structure(positions, conn, displacement = displacement)

## Exercice 2 : Console de deux poutres

On considère dans cet exercice une console formée par deux poutres:

In [ ]:
Image(filename = './Images/beams.png',  width=350, height=200)

1. Quel type d'élément est adapté à la résolution de ce problème ? Combien y a-t-il de degrés de liberté par élément ?

---

 **Solution:**

 ---



Il s'agit d'un problème 3D: le fléchissement et la torsion sont à prendre en compte par des poutres de Bernouilli 3D chacune avec 12 degrés de libertés. Sur chaque noeud on a un déplacement axial, une flèche horizontale et une verticale, un angle de torsion, un angle de fléchissement horizontal et un vertical: 6 degrés de libertés par noeud.

On considère que les poutres on le même module de Young $E$, inertie flexionnelle $I$ (dans toutes les directions), et constante de torsion $J$.

2. En utilisant les symétries du problème, déterminer les degrés de libertés nuls du système. Montrez que l'on peut réduire le nombre d'inconnues à trois. Est-il possible de déduire sans calcul la valeur d'un de ces degrés de liberté ?

---

 **Solution:**

 ---



Les deux encastrements enlèvent 6 degrés de libertés à chaque poutre. Les degrés de libertés restants sont au nombre de 6 par poutres, mais le nombre total de degrés libertés est plus faible car les poutres sont connectées. Le noeud d'encastrement a potentiellement 3 DDL de translation et 3 DDL de rotation. À cause de la symétrie du problème la rotation autour de $z$ doit être nulle.

Changeons de repère et prenons le repère $(O, x', y', z)$ où le plan $(O,x',z)$ correspond au plan de symétrie du problème. Il en découle que le déplacement selon $y'$ doit être nul (selon la condition de symétrie), la rotation selon $x'$ doit être nulle. Il reste donc:

   - Rotation selon $y'$
   - Translation selon $x'$
   - Translation selon $z$

Étant donné le cas de chargement, et l'hypothèse des petites déformations, on peut dire que la translation selon $x'$ est nulle. Les degrés de libertés non-nuls pour la poutre $i$ sont:

   - Déplacement vertical $v_i$
   - Rotation de torsion $\theta_i^T$
   - Rotation de flexion verticale $\theta_i$

Notez que les rotations $\theta_i^T$ et $\theta_i$ ne sont pas indépendantes.

3. Définir les matrices de rigidité locales $k_1$ et $k_2$, en se limitant aux degrés de libertés non-nuls.

---

 **Solution:**

 ---



Voici les matrices de rigidité des poutres $k_1$ et $k_2$ (dans les axes locaux):
    
$$
\begin{equation*}
k_1 \begin{pmatrix}
v_1 \\
\theta_1^T \\
\theta_1 \\
\end{pmatrix} = \begin{pmatrix}
12EI/L^3 & 0 & - 6EI/L^2  \\
0 & GJ/L & 0 \\ - 6EI/L^2 & 0 & 4EI/L
\end{pmatrix} \begin{pmatrix}
v_1 \\
\theta_1^T \\
\theta_1 \\
\end{pmatrix}
\end{equation*}
$$

et pour $k_2$

$$
\begin{equation*}
k_2 \begin{pmatrix}
v_2 \\
\theta_2^T \\
\theta_2 \\
\end{pmatrix} = \begin{pmatrix}
12EI/L^3 & 0 & -6EI/L^2  \\
0 & GJ/L & 0 \\
-6EI/L^2 & 0 & 4EI/L
\end{pmatrix} \begin{pmatrix}
v_2 \\
\theta_2^T \\
\theta_2 \\
\end{pmatrix}
\end{equation*}
$$

Posons $E = G = I = J = L = 1$ pour la simplicité. Il est également plus simple de considérer comme axes globaux le système d'axe $(O, x', y', z)$ défini par la figure ci-dessous:

In [ ]:
Image(filename = './Images/beams_symetry.png',  width=250, height=200)

4. Assembler la matrice de rigidité globale $K$.

On applique une matrice de rotation de $\frac{-\pi}{4}$ à la matrice de rigidité $k_1$:

$$
\begin{equation*}
R  = \begin{pmatrix}
1 & 0 & 0 \\
0 & \frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2}\\
0 & -\frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2}
\end{pmatrix}
\end{equation*}
\begin{equation*}
K_1 = R^Tk_1R = \begin{pmatrix}12 & - 3 \sqrt{2} & - 3 \sqrt{2}\\- 3 \sqrt{2} & \frac{5}{2} & \frac{3}{2}\\- 3 \sqrt{2} & \frac{3}{2} & \frac{5}{2}\end{pmatrix}
\end{equation*}
$$

On applique une matrice de rotation de $\frac{\pi}{4}$ à la matrice de rigidité $k_2$:

$$
\begin{equation*}
R  = \begin{pmatrix}
1 & 0 & 0 \\
0 & \frac{\sqrt{2}}{2} & -\frac{\sqrt{2}}{2}\\
0 & \frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2}
\end{pmatrix}
\end{equation*}
$$
    
Soit $K_2$ la matrice $k_2$ dans les axes globaux:
    
$$
\begin{equation*}
K_2 = R^Tk_2R = \begin{pmatrix}12 & 3 \sqrt{2} & - 3 \sqrt{2}\\3 \sqrt{2} & \frac{5}{2} & - \frac{3}{2}\\- 3 \sqrt{2} & - \frac{3}{2} & \frac{5}{2}\end{pmatrix}
\end{equation*}
$$

On peut directement faire la somme de $K_1$ et $K_2$ pour obtenir une matrice 3$\times$3. Par contre, on sait que l'on ne doit avoir que deux degrés de libertés (translation selon $z$ et rotation autour de $y'$), donc la matrice globale doit être une 2$\times$2. Projeté dans le système d'axes $(Ox'y'z)$, le vecteur d'inconnue $(v_i, \theta_i^T, \theta_i)$ devient $(v_z, \theta_{x'}, \theta_{y'})$. On sait par symétrie que $\theta_{x'} = 0$, on peut donc ignorer les lignes et colonnes centrales de $K_1$ et $K_2$.
    
$$
\begin{equation*}
K = \begin{pmatrix}
24 & -6\sqrt{2}\\
-6\sqrt{2} & 4
\end{pmatrix}
\end{equation*}
$$

5. Assembler le vecteur des forces nodales $F$.

---

 **Solution:**

 ---



L'assemblage du vecteur forces est trivial:
    
$$
\begin{equation}
f = \begin{pmatrix}
-P\\
0
\end{pmatrix}
\end{equation}
$$

6. Résoudre le système d'équations.

---

 **Solution:**

 ---



La solution est:
    $\begin{pmatrix}
        v_z\\
        \theta_{y'}
      \end{pmatrix} = \begin{pmatrix}- \frac{5 P}{48}\\- \frac{\sqrt{2} P}{8}\end{pmatrix}
   $